In [1]:
import pyspark

In [2]:
conf = pyspark.SparkConf()
conf.setMaster('spark://spark-master:7077')
spark_context = pyspark.SparkContext(conf=conf)

In [3]:
# Создаются данные для вычислений. range возвращает generator, а не лист =)
big_list = range(10000)

# Создаётся RDD-структура.
# Вызов parallelize превращает массив в распределённый для дальнейшей работы в кластере.
# Второй аргумент 2 — это количество частей, на которые делятся данные.
rdd = spark_context.parallelize(big_list, 2)

# Создаётся фильтр только по чётным числам. lambda-функция детерминированная.
odds = rdd.filter(lambda x: x % 2 != 0)

# Спрашиваем кластер первые 5 результатов работы.
print(odds.take(5))

[1, 3, 5, 7, 9]


In [3]:
# Открытие текстового файла и превращение его в RDD.
# К докеру со spark-worker примаунчена папка /user-data с вашего диска, в которой лежат скачанные данные.
data = spark_context.textFile('/home/developer/data/combined/reviews.csv')

# Берём первую строку — для дальнейшей фильтрации. Первая строка отвечает за заголовки, они нас не интересуют.
header_data = data.first()

# Начались вычисления. Первая функция пропускает все данные, кроме первой строки с заголовками.
# Функция map разделяет строку на колонки, используя разделитель `,`.
result_data = data \
    .filter(lambda row: row != header_data) \
    .map(lambda line: line.split(","))

result_data.take(10) 

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 4 times, most recent failure: Lost task 0.3 in stage 0.0 (TID 3, 172.20.0.4, executor 0): java.io.FileNotFoundException: File file:/home/developer/data/combined/reviews.csv does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:666)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:987)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:656)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:454)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:146)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:347)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:899)
	at org.apac

In [ ]:
result_data = result_data.filter(lambda line: len(line) > 5) 
result_data.take(10)

In [ ]:
# Отфильтровываем только числа
result_data = result_data.filter(lambda line: line[4].isdigit())
result_data.take(10)

In [ ]:
result_data = result_data.map(lambda line: (line[1], int(line[4])))
result_data.take(10)

In [ ]:
result_data = result_data.map(lambda line: (line[0], (line[1], 1))) 
result_data.take(10)

In [ ]:
result_data.take(10)

In [ ]:
# Подсчет суммы reduce функцией
result_data = result_data.reduceByKey(lambda val1, val2: (val1[0] + val2[0], val1[1] + val2[1]))
result_data.take(10)

In [ ]:
# Подсчет среднего арифметического
result_data = result_data.mapValues(lambda x: x[0] / x[1]) 
result_data.take(10)

In [ ]:
# Сохранение результата в файл
result_data.coalesce(1).saveAsTextFile('/user-data/combined/results') 